# Preparação dos Dados

## Imports e Inicialização

In [488]:
#pip install pandas numpy matplotlib plotly nbformat

In [489]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

file_path = './PEDE_PASSOS_DATASET_FIAP.csv'
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True
df_original = pd.read_csv(file_path, delimiter=';')

In [490]:
df_original.head(3)

### Ajustes e Transformações

Primeiramente, vamos separar os dados da coluna 'FASE_TURMA_2020' em duas colunas: 'FASE_2020' e 'TURMA_2020'

In [491]:
df_original[['FASE_2020', 'TURMA_2020']] = df_original['FASE_TURMA_2020'].str.extract(r'(.{1})(.{1})')
df_original.drop('FASE_TURMA_2020', axis=1, inplace=True)

In [492]:
df_original['ID_ALUNO']=df_original['NOME'].str.split('-').str[1].astype(int)
#df_original.set_index('ID_ALUNO', inplace=True)
#df_original.drop('NOME', axis=1, inplace=True)

In [493]:
df_original.head()

In [494]:
df_original.columns

Para facilitar a análise dos dados, vamos selecionar apenas as colunas que são comuns a todos os anos.

In [495]:
df_novo = df_original[['ID_ALUNO','NOME','FASE_2020','TURMA_2020','PONTO_VIRADA_2020','INDE_2020','PEDRA_2020','IAA_2020','IEG_2020','IPS_2020','IDA_2020','IPP_2020','IPV_2020','IAN_2020',
'FASE_2021','TURMA_2021','PONTO_VIRADA_2021','INDE_2021','PEDRA_2021','IAA_2021','IEG_2021','IPS_2021','IDA_2021','IPP_2021','IPV_2021','IAN_2021',
'FASE_2022','TURMA_2022','PONTO_VIRADA_2022','INDE_2022','PEDRA_2022','IAA_2022','IEG_2022','IPS_2022','IDA_2022','IPP_2022','IPV_2022','IAN_2022']]
df_novo

Observa-se que temos colunas atribuidas por ano, vamos separar esse DataFrame em 3 DataFrames por colunas referentes a cada ano, usando o "ID_ALUNO" e o "NOME" como chave

In [496]:
df_2020 = pd.concat([df_novo[['ID_ALUNO','NOME']], df_novo.filter(regex='_2020')], axis=1)
df_2021 = pd.concat([df_novo[['ID_ALUNO','NOME']], df_novo.filter(regex='_2021')], axis=1)
df_2022 = pd.concat([df_novo[['ID_ALUNO','NOME']], df_novo.filter(regex='_2022')], axis=1)

Agora iremos retirar as linhas que possuem somente valores nulos (NaN)

In [497]:
df_2020_filtrado = df_2020[df_2020['FASE_2020'].notnull()]
df_2021_filtrado = df_2021[df_2021['FASE_2021'].notnull()]
df_2022_filtrado = df_2022[df_2022['FASE_2022'].notnull()]

Agora vamos remover o sufixo das colunas e criar uma nova coluna 'ANO_PESQUISA'

In [498]:
df_2020_filtrado.columns = df_2020_filtrado.columns.str.removesuffix("_2020")
df_2021_filtrado.columns = df_2021_filtrado.columns.str.removesuffix("_2021")
df_2022_filtrado.columns = df_2022_filtrado.columns.str.removesuffix("_2022")


In [499]:
df_2020_filtrado.insert(loc=0, column='ANO_PESQUISA', value=2020)
df_2021_filtrado.insert(loc=0, column='ANO_PESQUISA', value=2021)
df_2022_filtrado.insert(loc=0, column='ANO_PESQUISA', value=2022)

In [500]:
print(df_2020_filtrado.columns)
print(df_2021_filtrado.columns)
print(df_2022_filtrado.columns)

In [501]:
df_2020_filtrado.info()

In [502]:
df_final = pd.concat([df_2020_filtrado, df_2021_filtrado, df_2022_filtrado], axis=0, ignore_index=True)

In [503]:
df_final

In [504]:
df_final.info()

In [505]:
df_final.iloc[680]

In [506]:
df_final.drop(680, inplace=True)

In [507]:
df_final.iloc[759]

In [508]:
df_final.replace('#NULO!', np.nan, inplace=True)

In [509]:
df_final[['INDE', 'IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP','IPV']] = df_final[['INDE', 'IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP','IPV']].astype(float)

In [510]:
df_final['IS_PONTO_VIRADA'] = df_final['PONTO_VIRADA'].map({'Sim': True, 'Não': False})
df_final['IS_PONTO_VIRADA'] = df_final['IS_PONTO_VIRADA'].astype(bool)
df_final['SYMBOL'] = df_final.IS_PONTO_VIRADA.apply(lambda x: 17 if x == True else 1)

In [511]:
df_final.info()

In [512]:
df_final[['ANO_PESQUISA','ID_ALUNO','FASE']] = df_final[['ANO_PESQUISA','ID_ALUNO','FASE']].astype(int)

In [513]:
df_final.info()

In [514]:
df = df_final.round(2)

In [515]:
df.head()

## Novas métricas

Vamos calcular as médias por ano dos alunos, agora que o dataframe está limpo para análise

In [516]:
df_media_ano_fase = df.groupby(['ANO_PESQUISA', 'FASE'])[['INDE', 'IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP','IPV']].mean().round(2)
df_media_ano_fase.columns = ['INDE_MEDIO_ANO_FASE', 'IAN_MEDIO_ANO_FASE', 'IDA_MEDIO_ANO_FASE', 'IEG_MEDIO_ANO_FASE', 'IAA_MEDIO_ANO_FASE', 'IPS_MEDIO_ANO_FASE', 'IPP_MEDIO_ANO_FASE', 'IPV_MEDIO_ANO_FASE']
df_media_ano_fase

In [517]:
df_media_ano_fase_turma = df.groupby(['ANO_PESQUISA', 'FASE', 'TURMA'])[['INDE', 'IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP','IPV']].mean().round(2)
df_media_ano_fase_turma.columns = ['INDE_MEDIO_ANO_FASE_TURMA', 'IAN_MEDIO_ANO_FASE_TURMA', 'IDA_MEDIO_ANO_FASE_TURMA', 'IEG_MEDIO_ANO_FASE_TURMA', 'IAA_MEDIO_ANO_FASE_TURMA', 'IPS_MEDIO_ANO_FASE_TURMA', 'IPP_MEDIO_ANO_FASE_TURMA', 'IPV_MEDIO_ANO_FASE_TURMA']
df_media_ano_fase_turma

In [518]:
join_media_ano_fase = pd.merge(
    left=df,
    right=df_media_ano_fase,
    how='left',
    left_on=['ANO_PESQUISA', 'FASE'],
    right_on=['ANO_PESQUISA', 'FASE'],
)
join_media_ano_fase

In [519]:
join_media_ano_fase_turma = pd.merge(
    left=join_media_ano_fase,
    right=df_media_ano_fase_turma,
    how='left',
    left_on=['ANO_PESQUISA', 'FASE','TURMA'],
    right_on=['ANO_PESQUISA', 'FASE', 'TURMA'],
)
join_media_ano_fase_turma

In [520]:
join_media_ano_fase_turma.to_csv('dados_finais.csv', index=False)

In [521]:
def gera_df_comparativo_aluno_medias(df):
    dataframes = {}
    for index, row in df.iterrows():
        ano_pesquisa = row['ANO_PESQUISA']
        data = {
            'Indice': ['IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP','IPV']*3,
            'Legenda': [f'Notas do Aluno ({ano_pesquisa})']*7 + [f'Médias da Fase ({ano_pesquisa})']*7 +[f'Médias da Turma ({ano_pesquisa})']*7,
            'Nota': [df['IAN'][index], df['IDA'][index], df['IEG'][index], df['IAA'][index], df['IPS'][index], df['IPP'][index], df['IPV'][index], df['IAN_MEDIO_ANO_FASE'][index], df['IDA_MEDIO_ANO_FASE'][index], df['IEG_MEDIO_ANO_FASE'][index], df['IAA_MEDIO_ANO_FASE'][index], df['IPS_MEDIO_ANO_FASE'][index], df['IPP_MEDIO_ANO_FASE'][index], df['IPV_MEDIO_ANO_FASE'][index], df['IAN_MEDIO_ANO_FASE_TURMA'][index], df['IDA_MEDIO_ANO_FASE_TURMA'][index], df['IEG_MEDIO_ANO_FASE_TURMA'][index], df['IAA_MEDIO_ANO_FASE_TURMA'][index], df['IPS_MEDIO_ANO_FASE_TURMA'][index], df['IPP_MEDIO_ANO_FASE_TURMA'][index], df['IPV_MEDIO_ANO_FASE_TURMA'][index]]
        }
        novo_df = pd.DataFrame(data)
        dataframes[ano_pesquisa] = novo_df
    return dataframes

In [522]:
def gera_df_comparativo_anos(df):
    data = {
        'Indice': [],
        'Legenda': [],
        'Nota': []
    }
    for index, row in df.iterrows():
        data['Indice'] += ['IAN', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP','IPV']
        data['Legenda'] += [f"Notas do Aluno ({df['ANO_PESQUISA'][index]})"]*7
        data['Nota'] += [df['IAN'][index],df['IDA'][index],df['IEG'][index],df['IAA'][index],df['IPS'][index],df['IPP'][index],df['IPV'][index]]
    novo_df = pd.DataFrame(data)
    return novo_df

In [523]:
df_copy = join_media_ano_fase_turma.copy()

In [524]:
df1 = df_copy[(df_copy['ID_ALUNO'] == 8)]

In [525]:
teste1 = gera_df_comparativo_aluno_medias(df1)
teste2 = gera_df_comparativo_anos(df1)

In [526]:
for index, key in enumerate(teste1):
    fig1 = px.line_polar(teste1[key], r="Nota", theta="Indice", color="Legenda",
                        line_close=True, title='Comparativo das Notas do Aluno com as Médias ('+str(key)+')', markers=True, range_r=[0,10],  start_angle=0, direction='counterclockwise')
    fig1.show()

In [535]:
fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'polar'}]*3],
                    subplot_titles=('2020', '2021', '2022'), column_titles=('2020', '2021', '2022'), x_title="Evolução do Aluno por Ano")
for index, key in enumerate(teste1):
    fig1 = px.line_polar(teste1[key], r="Nota", theta="Indice", color="Legenda", line_close=True, title="2020", markers=True, range_r=[0,10], start_angle=0, direction='counterclockwise')
    fig.add_traces(list(fig1.select_traces()),1,index+1)


fig.update_layout(
    showlegend=False
)

fig.show()

In [528]:
# df_melted = df1.melt(id_vars='ANO_PESQUISA', value_vars=['INDE', 'INDE_MEDIO_ANO_FASE', 'INDE_MEDIO_ANO_FASE_TURMA'], var_name='TIPO', value_name='NOTA')
# df_melted

In [529]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter'}, {'type': 'polar'}]],
                    subplot_titles=('INDE', 'Índices'), column_titles=('INDE', 'Índices'), x_title="Evolução do Aluno por Ano")
fig1 = px.line(df1, x="ANO_PESQUISA", y="INDE", markers=True, template="plotly_white",  symbol='SYMBOL')
fig2 = px.line_polar(teste2, r="Nota", theta="Indice", color="Legenda",
                    line_close=True, markers=True, range_r=[0,10], start_angle=0, direction='counterclockwise')

fig.add_traces(list(fig1.select_traces()),1,1)
fig.add_traces(list(fig2.select_traces()),1,2)
fig.show()

In [530]:
anos = df1['ANO_PESQUISA'].values
inde = df1['INDE'].values

plt.figure(figsize=(10, 5))
plt.plot(anos, inde, marker='o', linestyle='-', color='b', label='INDE')
plt.xticks(anos)
plt.xlabel('Ano')
plt.ylabel('INDE')
plt.title(f'Evolução do INDE para o aluno 5')
plt.legend()

# Cálculo da diferença percentual e anotação no gráfico
for i in range(1, len(anos)):
    diff_percent = ((inde[i] - inde[i-1]) / inde[i-1]) * 100
    diff_text = f"+{diff_percent:.2f}%" if diff_percent > 0 else f"{diff_percent:.2f}%"
    plt.text(anos[i], inde[i], diff_text, fontsize=9, verticalalignment='bottom', horizontalalignment='right')

In [531]:
df1